In [ ]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Salman2020Do_R50", "imagenet", batch_size=256)
# model, dl_train, dl_eval = load_torchvision_experiment("vgg16")

dl_eval.batch_size = 256

In [ ]:
from ulib.attack import StopCriteria
from ulib.attacks.uapgd import MIPGD, UAPGD, ValueScheduler
from ulib.pert_module import PertModule
from torch.optim.lr_scheduler import StepLR

stop = StopCriteria(max_epochs=20, max_time=60 * 60)
torch_attack = MIPGD(model, eps=16 / 255, alpha=0.1, gamma=0)
pert_model = PertModule(model, data_shape=dl_train.get_tensor(0).shape[1:], eps=16 / 255)
alpha_sched = ValueScheduler(init_value=0.1, sched_cls=StepLR, step_size=5, gamma=0.2)


attack = UAPGD(
    pert_model=pert_model,
    inner_attack=torch_attack,
    alpha_sched=alpha_sched,
    sched_on_batch=False,
    eval_freq=30,
    eval_on_batch=True,
    logging_enable=True,
)

pert = attack.fit(dl_train, dl_eval, stop)

In [ ]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)